# SF Assessment data

In [259]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [260]:
housing = pd.read_csv('Historic_Secured_Property_Tax_Rolls.csv')

Fraction of assessments in most common class

In [261]:
housing['Property Class Code'] = housing['Property Class Code'].fillna('NaN')
code_counts = housing['Property Class Code'] .value_counts()
max_code = code_counts.max().astype(float)
print(max_code/code_counts.sum())

0.470725322714


Median assessed improvement value

In [262]:
sorted_housing = housing.sort_values('Closed Roll Fiscal Year', ascending = False)
sorted_housing = sorted_housing.drop_duplicates(subset='Block and Lot Number')
sorted_housing['Closed Roll Assessed Improvement Value'].fillna(0, inplace = True)
assessments = sorted_housing[sorted_housing['Closed Roll Assessed Improvement Value']>0.0]
med_assessments = assessments['Closed Roll Assessed Improvement Value'].median()
print('Median assessed improvements: $%f'%med_assessments)

Median assessed improvements: $209240.000000


Average improvement value by neighborhood

In [263]:
neighborhoods = sorted_housing.groupby('Neighborhood Code')
Max = neighborhoods['Closed Roll Assessed Improvement Value'].mean().max()
Min = neighborhoods['Closed Roll Assessed Improvement Value'].mean().min()
print(Max, Min, Max-Min)

(5013197.0481682494, 132027.88435788194, 4881169.1638103677)


Yearly growth rate of land values

In [264]:
values = housing[['Closed Roll Fiscal Year','Closed Roll Assessed Land Value']]
values = values[values['Closed Roll Assessed Land Value']>0].dropna()
Y_ = np.log(values['Closed Roll Assessed Land Value']).reshape(-1,1)
t_ = np.asarray(values['Closed Roll Fiscal Year']-2007).reshape(-1,1)

regr = LinearRegression(normalize = True, n_jobs = -1)
regr.fit(t_,Y_)
coeffs = regr.coef_
intercept = regr.intercept_
score = regr.score(t_,Y_)
print(coeffs, intercept)

(array([[ 0.04188026]]), array([ 11.67322951]))


Neighborhood areas

In [265]:
Locations = sorted_housing[['Neighborhood Code','Location']]
Locations = Locations.replace([np.inf, -np.inf],np.nan).dropna()
XLocs = [float(l.split(',')[0].strip('(')) for l in Locations['Location']]
YLocs = [float(l.split(',')[1].strip(')')) for l in Locations['Location']]
Locations['Lat']=XLocs
Locations['Long']=YLocs

Neighborhood_size = Locations.groupby('Neighborhood Code')
Lat_dim = Neighborhood_size['Lat'].std()
Long_dim = Neighborhood_size['Lat'].std()
area = np.pi*(Lat_dim/2)*(Long_dim/2)
biggest_neighborhood = Neighborhood_size.get_group(area.idxmax())
mean_lat = biggest_neighborhood['Lat'].mean()
mean_long = biggest_neighborhood['Long'].mean()
print(mean_lat,mean_long)
# conversion to kilometers at 37.72 is 1:110.991 for lat and 1:88.165 for long, see msi.nga.mil
Lat_dim_km = biggest_neighborhood['Lat'].std()*110.991
Long_dim_km = biggest_neighborhood['Long'].std()*88.165
biggest_area = np.pi*(Lat_dim_km/2)*(Long_dim_km/2)
print(biggest_area)

(37.721324864785458, -122.38002572569654)
0.758988541274


Comparing average units for buildings built pre and post 1950

In [266]:
units = housing[['Closed Roll Fiscal Year','Block and Lot Number','Number of Units',
                 'Year Property Built']].sort_values('Closed Roll Fiscal Year', 
                                                     ascending = True).drop_duplicates(subset='Block and Lot Number')
# the oldest property in SF was built in 1776. The dataset should not include properties built after 2015.
keep = (units['Number of Units']>0) & (units['Year Property Built']<2016) & (units['Year Property Built']>1776)
units = units[keep]
average_pre1950 = units['Number of Units'][units['Year Property Built']<1950].mean()
average_post1950 = units['Number of Units'][units['Year Property Built']>=1950].mean()
print(abs(average_pre1950-average_post1950))

0.41985685735


Bedrooms per unit by zipcode

In [267]:
bedrooms = sorted_housing[(sorted_housing['Number of Bedrooms']>0) & (sorted_housing['Number of Units']>0)]
mean_bedrooms = bedrooms.groupby('Zipcode of Parcel')['Number of Bedrooms'].mean()
mean_units = bedrooms.groupby('Zipcode of Parcel')['Number of Units'].mean() 
ratios = (mean_bedrooms/mean_units)
print(ratios.max())

3.80756013746


Total property area to total lot area, by zipcode

In [268]:
built_up = sorted_housing[sorted_housing['Lot Area']>0]
property_area = built_up.groupby('Zipcode of Parcel')['Property Area in Square Feet'].sum()
lot_area = built_up.groupby('Zipcode of Parcel')['Lot Area'].sum()
built_ratio = property_area/lot_area
print(built_ratio.max())

12.0603296061
